# Импорт необходимых библиотек

In [1]:
!pip install --proxy 10.0.0.1:3128 gensim emoji pandas nltk pymorphy2

Using legacy 'setup.py install' for emoji, since package 'wheel' is not installed.
Using legacy 'setup.py install' for docopt, since package 'wheel' is not installed.
    Running setup.py install for emoji: started
    Running setup.py install for emoji: finished with status 'done'
    Running setup.py install for docopt: started
    Running setup.py install for docopt: finished with status 'done'


You should consider upgrading via the 'c:\users\admin\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


In [55]:
import pandas as pd
import nltk
from nltk.corpus import brown
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
from gensim.models import Word2Vec as wv
from gensim.models import KeyedVectors
import pymorphy2
from pymorphy2 import MorphAnalyzer
import emoji
import numpy as np
from random import randint

In [3]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [4]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [5]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


True

In [6]:
nltk.download('averaged_perceptron_tagger_ru')

[nltk_data] Downloading package averaged_perceptron_tagger_ru to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger_ru.zip.


True

In [7]:
nltk.download('universal_tagset')

[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\universal_tagset.zip.


True

# Датасет ВК

## Загрузка датасета

In [8]:
colnames = ['comment',]

In [9]:
comments = pd.read_csv('datasets/vk.csv', names=colnames, header=0, sep='\t')

In [10]:
comments

comment
0                                                  ['😭']
1      ['хранить', 'господь', 'неравнодушный', 'челов...
2      ['евгений', 'еслиб', 'бог', 'такой', 'допускать']
3              ['кирилл', 'полностью', 'согласный', '😢']
4             ['кирилл', 'бог', 'путь', 'неисповедимый']
...                                                  ...
50779                              ['с', 'фамилия', '🙄']
50780  ['наш', 'день', 'повезти', 'мужик', 'фамилия',...
50781               ['чудо', 'чудной', 'диво', 'дивный']
50782  ['пригреться', 'уснуть', 'свариться', 'лягушка...
50783  ['добрый', 'день', 'вполне', 'возможно', 'пере...

[50784 rows x 1 columns]

In [11]:
def str_to_list(i):
    new_list = eval(i)
    return new_list

In [12]:
comments['comment'] = comments['comment'].apply(str_to_list)

### Выделение эмодзи флагов

In [13]:
def split(s):
    return[char for char in s]

reg_ident = ['🇷', '🇺', '🇦', '🇸', '🇧', '🇨', '🇩', '🇪', '🇫', '🇬', '🇭', '🇮', '🇯', '🇰', '🇱', '🇲', '🇳', '🇴', '🇵', '🇶', '🇸', '🇹', 'v', '🇼', '🇽', '🇾', '🇿']

def get_flags(doc):
    for item in doc:
        new_item = []
        for c in split(item):
            if c in reg_ident:
                if len(new_item) < 2:
                    new_item.append(c)
        if len(new_item) == 2 and new_item[0] != new_item[1]:
            try:
                i = doc.index(item)
                doc[i] = new_item[0] + new_item[1]
            except ValueError:
                doc[i] = ''
    return doc

In [14]:
comments['comment'] = comments['comment'].apply(get_flags)

In [216]:
comments

comment
0                                                    [😭]
1       [хранить, господь, неравнодушный, человек, 🙏, 🕊]
2                [евгений, еслиб, бог, такой, допускать]
3                      [кирилл, полностью, согласный, 😢]
4                     [кирилл, бог, путь, неисповедимый]
...                                                  ...
50779                                    [с, фамилия, 🙄]
50780  [наш, день, повезти, мужик, фамилия, но, суть,...
50781                       [чудо, чудной, диво, дивный]
50782  [пригреться, уснуть, свариться, лягушка, так, ...
50783  [добрый, день, вполне, возможно, перевозчик, з...

[50784 rows x 1 columns]

In [217]:
true_vk_data = comments['comment'].values.tolist()

### Добавление тегов из universal_tagset. Для этого нужно переименовать _taggers/universal_tagset/ru-rnс.map_ в _taggers/universal_tagset/ru-rnc-new.map_

In [16]:
def tagger(doc):
    tagged_tokens = []
    try:
        tagged_doc = nltk.pos_tag(doc, tagset='universal', lang = 'rus')
        for p in tagged_doc:
            if emoji.is_emoji(p[0]):
                token = p[0] + '_SYM'
            else:
                token = p[0] + '_' + p[1]
            tagged_tokens.append(token)
    except KeyError:
        print('Возникла ошибка формата токена, запись пропущена!')
    return(tagged_tokens)

In [17]:
comments_tagged = pd.DataFrame()

In [19]:
comments_tagged['tokens'] = comments['comment'].apply(tagger)

['ут', 'чк', 'м', 'ик', 'н', 'ки', 'тны', 'далее', 'м', 'нт', 'так', 'ы', 'ют', 'ук', 'ины', 'ближн', 'г', 'в', 'так', 'кит', 'так', 'л', 'ь', 'зный', 'далее', 'з', 'так', 'ль', 'тв', 'м', 'так', 'г', 'чт', 'к', 'мль', 'н', 'дит', 'п', 'далее', 'лп', 'м', 'п', 'нт', 'г', 'н', 'чт', 'ия', 'так', 'л', 'язвить', 'й', 'вн', 'шни', 'г', 'з', 'эт', 'зн', 'ч', 'так', 'чт', 'н', 'ци', 'н', 'льн', 'б', 'з', 'п', 'н', 'ть', 'ия', 'н', 'дит', 'п', 'далее', 'г', 'з', 'й', 'ия', 'ки', 'вл', 'ти', 'далее', 'лжный', 'п', 'инять', 'чны', 'м', 'ы', 'з', 'щит', 'так', 'ны']
Тут ошибка формата!!
['ут', 'чк', 'тны', 'далее', 'м', 'нт', 'ющи', 'ук', 'ины', 'ближн', 'г', 'в', 'так', 'кит', 'вид', 'так', 'ль', 'тв', 'так', 'так', 'м', 'чт', 'ия', 'н', 'дит', 'п', 'далее', 'п', 'так', 'льный', 'нт', 'л', 'м', 'м', 'ик', 'н', 'й', 'зв', 'дки', 'далее', 'м', 'нты', 'п', 'нт', 'г', 'н', 'г', 'ят', 'н', 'м', 'так', 'м', 'чт', 'широта', 'зн', 'ют', 'п', 'ия', 'ч', 'нь', 'мн', 'г', 'эт', 'явля', 'так', 'г', 'з', 'й

In [20]:
comments_tagged

tokens
0                                                [😭_SYM]
1      [хранить_VERB, господь_NOUN, неравнодушный_ADJ...
2      [евгений_ADJ, еслиб_NOUN, бог_NOUN, такой_PRON...
3      [кирилл_VERB, полностью_ADV, согласный_VERB, 😢...
4      [кирилл_VERB, бог_NOUN, путь_NOUN, неисповедим...
...                                                  ...
50779                       [с_ADP, фамилия_NOUN, 🙄_SYM]
50780  [наш_PRON, день_NOUN, повезти_VERB, мужик_NOUN...
50781      [чудо_NOUN, чудной_ADJ, диво_ADV, дивный_ADJ]
50782  [пригреться_VERB, уснуть_VERB, свариться_VERB,...
50783  [добрый_ADJ, день_NOUN, вполне_ADV, возможно_P...

[50784 rows x 1 columns]

### Формирование списка списков

In [21]:
vk_data = comments_tagged['tokens'].values.tolist()

In [61]:
vk_data[:3]

[['😭_SYM'],
 ['хранить_VERB',
  'господь_NOUN',
  'неравнодушный_ADJ',
  'человек_NOUN',
  '🙏_SYM',
  '🕊_SYM'],
 ['евгений_ADJ', 'еслиб_NOUN', 'бог_NOUN', 'такой_PRON', 'допускать_VERB']]

## Создание тестового датафрейма для переобучения

In [ ]:
from pandas.core.common import SettingWithCopyWarning

In [ ]:
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [157]:
test_df = comments_tagged[:100000]

In [158]:
for i in range(len(test_df)):
    flag = randint(1, 54)
    if flag == 1:
        test_df['tokens'].iloc[i] = ['федерация_NOUN', '👍_SYM', 'правда_NOUN', '🇷🇺_SYM', 'россия_NOUN', 'российский_ADJ', 'русский_ADJ']
    elif flag == 2:
        test_df['tokens'].iloc[i] = ['русский_ADJ' '👍_SYM', 'российский_ADJ', '🇷🇺_SYM', 'россия_NOUN']
    elif flag == 3:
        test_df['tokens'].iloc[i] = ['правильно_ADV', '👍_SYM', 'молодец_NOUN', 'класс_NOUN', 'правда_NOUN' 'красавчик_NOUN']
    elif flag == 4:
        test_df['tokens'].iloc[i] = ['правда_NOUN', 'правильно_ADV', '😎_SYM', 'красавчик_NOUN', '👍_SYM', '👌_SYM', '😉_SYM', '🇷🇺_SYM', 'молодец_NOUN', 'правда_NOUN'] 
    elif flag == 5:
        test_df['tokens'].iloc[i] = ['смешно_ADV', '🤣_SYM', 'смеяться_VERB', '😄_SYM', 'смех_NOUN', 'угар_NOUN', '😅_SYM'] 
    elif flag == 6:
        test_df['tokens'].iloc[i] = ['смешно_ADV', 'смешной_ADJ', 'смех_NOUN', 'ржач_NOUN', '🤣_SYM', 'красавчик_NOUN'] 
    elif flag == 7:
        test_df['tokens'].iloc[i] = ['приятно_ADV', 'весело_ADV', '😊_SYM', 'улыбка_NOUN', '😄_SYM', '😅_SYM', 'радость_NOUN', '🤗_SYM', 'смеяться_VERB'] 
    elif flag == 8:
        test_df['tokens'].iloc[i] = ['весело_ADV', '😊_SYM', 'улыбка_NOUN', '😄_SYM', 'правда_NOUN', 'одобрение_NOUN', 'одобрять_VERB'] 
    elif flag == 9:
        test_df['tokens'].iloc[i] = ['красиво_ADV', '😍_SYM', 'красота_NOUN', 'любовь_NOUN', 'прекрасно_ADV'] 
    elif flag == 10:
        test_df['tokens'].iloc[i] = ['любить_VERB', 'красиво_ADV', '😍_SYM', 'влюблённость_NOUN', 'прелесть_NOUN', 'красавчик_NOUN'] 
    elif flag == 11:
        test_df['tokens'].iloc[i] = ['неуверенно_ADV', '❓_SYM', 'вопрос_NOUN', 'неуверенность_NOUN', 'спрашивать_VERB'] 
    elif flag == 12:
        test_df['tokens'].iloc[i] = ['фейспалм_NOUN', 'глупо_ADV', '🤦_SYM', 'глупость_NOUN', 'дурак_NOUN', 'тупость_NOUN'] 
    elif flag == 13:
        test_df['tokens'].iloc[i] = ['фейспалм_NOUN', 'тупо_ADV', '🤦_SYM', 'сомнение_NOUN', 'дурак_NOUN', 'тупость_NOUN'] 
    elif flag == 14:
        test_df['tokens'].iloc[i] = ['смешно_ADV', '😁_SYM', '🤣_SYM', 'смеяться_VERB', 'смех_NOUN', 'угар_NOUN'] 
    elif flag == 15:
        test_df['tokens'].iloc[i] = ['смешно_ADV', '😁_SYM', 'смешной_ADJ', 'смех_NOUN', 'ржач_NOUN', '🤣_SYM', 'красавчик_NOUN'] 
    elif flag == 16:
        test_df['tokens'].iloc[i] = ['вероятно_ADV', '🤔_SYM', 'неуверенно_ADV', '😨_SYM', '🤷_SYM', 'сомневаться_VERB', '😐_SYM', 'неуверенность_NOUN', 'сомнение_NOUN', 'вероятность_NOUN'] 
    elif flag == 17:
        test_df['tokens'].iloc[i] = ['фейспалм_NOUN', 'тупо_ADV', '🤦_SYM', 'сомнение_NOUN', '🤷_SYM', 'сомневаться_VERB', 'тупость_NOUN'] 
    elif flag == 18:
        test_df['tokens'].iloc[i] = ['сомнительно_ADV', '🤔_SYM', 'неуверенно_ADV', '🤷_SYM', 'сомневаться_VERB', '😐_SYM', 'неуверенность_NOUN', 'сомнение_NOUN', 'вероятность_NOUN'] 
    elif flag == 19:
        test_df['tokens'].iloc[i] = ['поздравление_NOUN', '👏_SYM', 'апплодисменты_NOUN', 'ура_NOUN', 'хлопать_VERB'] 
    elif flag == 20:
        test_df['tokens'].iloc[i] = ['грустно_ADV', '😢_SYM', 'грусть_NOUN', 'разочарование_NOUN', 'грустить_VERB', 'расстраиваться_VERB'] 
    elif flag == 21:
        test_df['tokens'].iloc[i] = ['грустно_ADV', '😢_SYM', 'грусть_NOUN', 'тоска_NOUN', 'грустить_VERB', 'расстраиваться_VERB'] 
    elif flag == 22:
        test_df['tokens'].iloc[i] = ['правильно_ADV', '🙏_SYM', 'мольба_NOUN', 'молиться_VERB', 'просьба_NOUN'] 
    elif flag == 23:
        test_df['tokens'].iloc[i] = ['православно_ADV', '🙏_SYM', 'мольба_NOUN', 'молиться_VERB', 'просьба_NOUN', 'просить_VERB'] 
    elif flag == 24:
        test_df['tokens'].iloc[i] = ['угарно_ADV', '😅_SYM', '😆_SYM', 'ржач_NOUN', 'смех_NOUN', , 'стёб_NOUN', 'стебать_VERB']
    elif flag == 25:
        test_df['tokens'].iloc[i] = ['забавно_ADV', '😉_SYM', 'подмигивать_VERB', 'молодец_NOUN', 'улыбка_NOUN', 'радость_NOUN'] 
    elif flag == 26:
        test_df['tokens'].iloc[i] = ['сильно_ADV', '💪_SYM', 'сила_NOUN', 'правда_NOUN', 'власть_NOUN'] 
    elif flag == 27:
        test_df['tokens'].iloc[i] = ['сильно_ADV', '💪_SYM', 'мощь_NOUN', 'сила_NOUN', 'правда_NOUN', 'власть_NOUN', '👍_SYM'] 
    elif flag == 28:
        test_df['tokens'].iloc[i] = ['ужасно_ADV', '😭_SYM', 'плачь_NOUN', 'грусть_NOUN', 'тоска_NOUN', 'плакать_VERB'] 
    elif flag == 29:
        test_df['tokens'].iloc[i] = ['плохо_ADV', '😭_SYM', 'плакать_VERB', 'грусть_NOUN', 'тоска_NOUN', 'плакать_VERB', 'рыдать_VERB'] 
    elif flag == 30:
        test_df['tokens'].iloc[i] = ['приятно_ADV', '😊_SYM', 'добро_NOUN', 'позитив_NOUN', 'радость_NOUN'] 
    elif flag == 31:
        test_df['tokens'].iloc[i] = ['плохо_ADV', 'негативно_ADV', '😡_SYM', 'негатив_NOUN', 'агрессия_NOUN', '😠_SYM', 'агрессивность_NOUN', 'зло_NOUN', 'злоба_NOUN'] 
    elif flag == 32:
        test_df['tokens'].iloc[i] = ['плохо_ADV', 'агрессивно_ADV', '😡_SYM', 'злоба_NOUN', '😠_SYM', 'злиться_VERB', '👿_SYM', 'злоба_NOUN', '😾_SYM',] 
    elif flag == 33:
        test_df['tokens'].iloc[i] = ['плохо_ADV', 'негативно_ADV', '😡_SYM', 'агрессия_NOUN', 'агрессивность_NOUN', 'злоба_NOUN'] 
    elif flag == 34:
        test_df['tokens'].iloc[i] = ['ужасно_ADV', '😤_SYM', 'агрессия_NOUN', '😠_SYM', 'агрессивность_NOUN', 'злоба_NOUN', 'ругаться_VERB',] 
    elif flag == 35:
        test_df['tokens'].iloc[i] = ['ужасно_ADV', 'негативно_ADV', '😤_SYM', 'ругань_NOUN', '😡_SYM', 'ругаться_VERB', '🤬_SYM', 'злоба_NOUN'] 
    elif flag == 36:
        test_df['tokens'].iloc[i] = ['ужасно_ADV', 'негативно_ADV', '😤_SYM', 'ругань_NOUN', '😡_SYM', 'зло_NOUN', 'ругать_VERB', 'ругаться_VERB', '🤬_SYM', 'материть_VERB'] 
    elif flag == 37:
        test_df['tokens'].iloc[i] = ['ругань_NOUN', '😡_SYM', 'зло_NOUN', 'ругать_VERB', 'ругаться_VERB', '🤬_SYM', 'материть_VERB'] 
    elif flag == 38:
        test_df['tokens'].iloc[i] = ['негативно_ADV', '💀_SYM', 'смерть_NOUN', 'негатив_NOUN', 'убийство_NOUN', 'убивать_VERB'] 
    elif flag == 39:
        test_df['tokens'].iloc[i] = ['правильно_ADV', '🤗_SYM', 'молодец_NOUN', 'правда_NOUN', 'красавчик_NOUN'] 
    elif flag == 40:
        test_df['tokens'].iloc[i] = ['смешно_ADV', '🤡_SYM', 'клоун_NOUN', 'насмешка_NOUN', 'насмехаться_VERB'] 
    elif flag == 41:
        test_df['tokens'].iloc[i] = ['клоунский_ADJ', 'угарно_ADV', '🤡_SYM', 'клоун_NOUN',  'насмешка_NOUN', 'насмехаться_VERB'] 
    elif flag == 42:
        test_df['tokens'].iloc[i] = ['смешно_ADV', 'клоун_NOUN', '🤡_SYM', 'ирония_NOUN', 'сарказм_NOUN', 'насмешка_NOUN', '😈_SYM', 'насмехаться_VERB'] 
    elif flag == 43:
        test_df['tokens'].iloc[i] = ['смешно_ADV', '🤡_SYM', 'клоун_NOUN', 'насмешка_NOUN', '😈_SYM', 'насмехаться_VERB', '💩_SYM'] 
    elif flag == 44:
        test_df['tokens'].iloc[i] = ['хреново_ADV', '💩_SYM', 'говно_NOUN', 'отстой_NOUN', 'хрень_NOUN']
    elif flag == 45:
        test_df['tokens'].iloc[i] = ['плохо_ADV', '💩_SYM', 'говно_NOUN', '🤢_SYM', 'ужас_NOUN'] 
    elif flag == 46:
        test_df['tokens'].iloc[i] = ['огонь_NOUN', '🔥_SYM', 'хорошо_ADV', 'огненно_ADV', 'молодец_NOUN'] 
    elif flag == 47:
        test_df['tokens'].iloc[i] = ['огонь_NOUN', '🔥_SYM', '😎_SYM', 'хорошо_ADV', 'огненно_ADV', 'молодец_NOUN'] 
    elif flag == 48:
        test_df['tokens'].iloc[i] = ['сомнение_NOUN', '😒_SYM', '😩_SYM', 'разочарование_NOUN', '😞_SYM', 'расстроен_ADV', 'расстроенность_NOUN', '😔_SYM'] 
    elif flag == 49:
        test_df['tokens'].iloc[i] = ['😩_SYM', '😞_SYM', 'расстроен_ADV', 'расстроенность_NOUN', '😔_SYM', 'разочарование_NOUN', '😒_SYM'] 
    elif flag == 50:
        test_df['tokens'].iloc[i] = ['кринж_NOUN', '😬_SYM', '🤧_SYM', 'плохо_ADV', 'огненно_ADV', 'молодец_NOUN'] 
    elif flag == 51:
        test_df['tokens'].iloc[i] = ['страх_NOUN', '🤧_SYM', '😰_SYM', 'страшно_ADV', '😳_SYM', 'шок_NOUN', 'бояться_VERB']
    elif flag == 52:
        test_df['tokens'].iloc[i] = ['свинья_NOUN', '🐷_SYM', '🐖_SYM', 'сало_ADV', 'сало_NOUN', '🐽_SYM', 'свин_NOUN', 'хохол_VERB']
    elif flag == 53:
        test_df['tokens'].iloc[i] = ['сало_NOUN', '🐖_SYM', '🐷_SYM', 'свинья_NOUN', 'хохол_NOUN', 'хохлятский_ADJ', 'украина_NOUN', 'украинский_ADJ']
    elif flag == 54:
        test_df['tokens'].iloc[i] = ['украина_NOUN', '🇺🇦_SYM', 'ципсо_NOUN', 'украинский_ADJ', 'хохлятский_ADJ']

C:\Users\Admin\AppData\Local\Temp\ipykernel_3508\1892931766.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['tokens'].iloc[i] = ['федерация_NOUN', '👍_SYM', 'правда_NOUN', '🇷🇺_SYM', 'россия_NOUN', 'российский_ADJ', 'русский_ADJ']
C:\Users\Admin\AppData\Local\Temp\ipykernel_3508\1892931766.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['tokens'].iloc[i] = ['русский_ADJ' '👍_SYM', 'российский_ADJ', '🇷🇺_SYM', 'россия_NOUN']
C:\Users\Admin\AppData\Local\Temp\ipykernel_3508\1892931766.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in t

In [159]:
test_df

tokens
0      [федерация_NOUN, 👍_SYM, правда_NOUN, 🇷🇺_SYM, р...
1      [русский_ADJ👍_SYM, российский_ADJ, 🇷🇺_SYM, рос...
2      [федерация_NOUN, 👍_SYM, правда_NOUN, 🇷🇺_SYM, р...
3      [правильно_ADV, 👍_SYM, молодец_NOUN, правда_NOUN]
4      [русский_ADJ👍_SYM, российский_ADJ, 🇷🇺_SYM, рос...
...                                                  ...
49995  [федерация_NOUN, 👍_SYM, правда_NOUN, 🇷🇺_SYM, р...
49996  [федерация_NOUN, 👍_SYM, правда_NOUN, 🇷🇺_SYM, р...
49997  [русский_ADJ👍_SYM, российский_ADJ, 🇷🇺_SYM, рос...
49998  [правильно_ADV, 👍_SYM, молодец_NOUN, правда_NOUN]
49999  [правильно_ADV, 👍_SYM, молодец_NOUN, правда_NOUN]

[50000 rows x 1 columns]

In [160]:
test_data = test_df['tokens'].values.tolist()

In [161]:
test_data[:3]

[['федерация_NOUN',
  '👍_SYM',
  'правда_NOUN',
  '🇷🇺_SYM',
  'россия_NOUN',
  'российский_ADJ',
  'русский_ADJ'],
 ['русский_ADJ👍_SYM', 'российский_ADJ', '🇷🇺_SYM', 'россия_NOUN'],
 ['федерация_NOUN',
  '👍_SYM',
  'правда_NOUN',
  '🇷🇺_SYM',
  'россия_NOUN',
  'российский_ADJ',
  'русский_ADJ']]

# Модель Word2Vec

## Инициализация готовой модели *(RusVectōrēs news_upos_skipgram_300_5_2019)*

### Загрузка датасета лента.ру и создание списка лента

In [63]:
lenta_read = pd.read_csv('lenta_df_lern.csv')

In [64]:
lenta_read.head(3)

Unnamed: 0                                               text
0           0  ['бой', 'сопоцкина', 'друскеник', 'закончиться...
1           1  ['министерство', 'народный', 'просвещение', 'в...
2           2  ['штабс', 'капитан', 'п', 'н', 'нестеров', 'де...

In [65]:
lenta_df = pd.DataFrame()

In [66]:
lenta_read = lenta_read[:150000]

In [67]:
lenta_df['tokens'] = lenta_read['text'].apply(str_to_list)

In [68]:
lenta_df['tokens'] = lenta_df['tokens'].apply(tagger)

['сегодня', 'ночью', 'м', 'год', 'жизнь', 'продолжительный', 'болезнь', 'свой', 'московский', 'квартира', 'трехпрудный', 'переулок', 'скончаться', 'королева', 'русский', 'романс', 'народный', 'артистка', 'россия', 'изабелла', 'данилович', 'юрьев', 'как', 'сообщать', 'риа', 'новость', 'прощание', 'певица', 'пройти', 'помещение', 'столичный', 'театр', 'эстрада', 'понедельник', 'январь', 'в', 'день', 'состояться', 'похороны', 'творческий', 'путь', 'изабелла', 'юрьев', 'начинаться', 'быть', 'год', 'с', 'дебют', 'прийти', 'успех', 'юрьев', 'войти', 'плеяда', 'тот', 'стоять', 'исток', 'современный', 'эстрадный', 'искусство', 'наш', 'страна', 'весь', 'долгий', 'жизнь', 'эстрада', 'последний', 'концерт', 'изабелла', 'данилович', 'состояться', 'середина', 'х', 'год', 'хранить', 'верность', 'романс', 'особый', 'место', 'репертуар', 'занимать', 'цыганский', 'романс', 'исполнение', 'который', 'юрьев', 'окрестить', 'белый', 'цыганка', 'мадам', 'вечный', 'аншлаг']
Тут ошибка формата!!
['сотрудник', 

['сегодня', 'белоруссия', 'м', 'год', 'жизнь', 'скончаться', 'известный', 'писатель', 'василь', 'быков', 'сообщать', 'интерфакс', 'как', 'сообщить', 'интерфакс', 'невестка', 'писатель', 'татьяна', 'быков', 'литератор', 'умереть', 'реанимационный', 'отделение', 'онкологический', 'больница', 'боровлян', 'минский', 'напомнить', 'некоторый', 'время', 'назад', 'быков', 'перенести', 'чехия', 'операция', 'удаление', 'раковый', 'опухоль', 'желудок', 'однако', 'развитие', 'болезнь', 'остановить', 'удаться', 'день', 'место', 'похороны', 'пока', 'определить', 'уточнять', 'агентство', 'василь', 'быков', 'родиться', 'июнь', 'год', 'город', 'бычок', 'ушацкий', 'район', 'витебский', 'область', 'учиться', 'скульптурный', 'отделение', 'витебский', 'художественный', 'училище', 'окончить', 'саратовский', 'пехотный', 'училище', 'в', 'год', 'будущий', 'писатель', 'вступить', 'ряд', 'красный', 'армия', 'воевать', 'второй', 'третий', 'украинский', 'фронт', 'пройти', 'действовать', 'армия', 'территория', 'рум

In [69]:
lenta_df.head(3)

tokens
0  [бой_NOUN, сопоцкина_NOUN, друскеник_NOUN, зак...
1  [министерство_NOUN, народный_ADJ, просвещение_...
2  [штабс_NOUN, капитан_NOUN, п_NOUN, н_VERB, нес...

In [70]:
lenta_data = lenta_df['tokens'].values.tolist()

### Инициализация модели и создание словаря на датасете лента.ру

In [235]:
model_vk_emoji = wv(vector_size=300, min_count=3, window=10)

In [236]:
model_vk_emoji.build_vocab(lenta_data)

In [237]:
len(model_vk_emoji.wv)

69181

In [238]:
model_vk_emoji.wv.vectors_lockf = np.ones(len(model_vk_emoji.wv))

In [239]:
model_vk_emoji.wv.intersect_word2vec_format('model.bin', lockf=1.0, binary=True)

In [240]:
model_vk_emoji.wv.most_similar('россия_NOUN')

[('будуть_VERB', 0.48997509479522705),
 ('геополитика_NOUN', 0.45983627438545227),
 ('беларусь_VERB', 0.457935631275177),
 ('европа_NOUN', 0.43242180347442627),
 ('русь_VERB', 0.42832908034324646),
 ('телеканал_VERB', 0.41772469878196716),
 ('аиф_NOUN', 0.4168393015861511),
 ('вестник_NOUN', 0.4098126292228699),
 ('глобализм_NOUN', 0.40060439705848694),
 ('спб_NOUN', 0.4003465175628662)]

### Дополнение словаря на тестовых данных и на данных вк и проверка того, что ничего не изменилось

In [200]:
model_vk_emoji.build_vocab(test_data, update=True)  # , update=True

In [201]:
len(model_vk_emoji.wv)

69184

In [202]:
model_vk_emoji.wv.similarity('россия_NOUN', '🇷🇺_SYM')

0.023919884

In [203]:
model_vk_emoji.wv.most_similar('россия_NOUN')

[('будуть_VERB', 0.48997509479522705),
 ('геополитика_NOUN', 0.45983627438545227),
 ('беларусь_VERB', 0.457935631275177),
 ('европа_NOUN', 0.43242180347442627),
 ('русь_VERB', 0.42832908034324646),
 ('телеканал_VERB', 0.41772469878196716),
 ('аиф_NOUN', 0.4168393015861511),
 ('вестник_NOUN', 0.4098126292228699),
 ('глобализм_NOUN', 0.40060439705848694),
 ('спб_NOUN', 0.4003465175628662)]

## Обучение модели на тестовых данных

In [204]:
model_vk_emoji.train(test_data, total_examples=model_vk_emoji.corpus_count, epochs=1)

(27050, 249920)

In [205]:
model_vk_emoji.wv.similarity('россия_NOUN', '🇷🇺_SYM')

0.7011695

In [206]:
model_vk_emoji.wv.most_similar('россия_NOUN')

[('🇷🇺_SYM', 0.701169490814209),
 ('будуть_VERB', 0.4873509109020233),
 ('геополитика_NOUN', 0.4638541340827942),
 ('беларусь_VERB', 0.4625330865383148),
 ('русь_VERB', 0.4434511959552765),
 ('европа_NOUN', 0.422006756067276),
 ('российский_ADJ', 0.41696539521217346),
 ('аиф_NOUN', 0.4140143394470215),
 ('вестник_NOUN', 0.4097059965133667),
 ('глобализм_NOUN', 0.4096096158027649)]

## Обучение модели на данных ВК

In [241]:
model_vk_emoji.build_vocab(vk_data, update=True)  # , update=True

In [242]:
len(model_vk_emoji.wv)

72357

In [243]:
model_vk_emoji.train(vk_data, total_examples=model_vk_emoji.corpus_count, epochs=1)

(357985, 393800)

In [244]:
model_vk_emoji.wv.similarity('россия_NOUN', '🐷_SYM')

0.49623203

In [250]:
model_vk_emoji.wv.most_similar('🐷_SYM')

[('♂️_SYM', 0.9745510816574097),
 ('👌_SYM', 0.9713121056556702),
 ('🐖_SYM', 0.9681888222694397),
 ('ципсо_NOUN', 0.9677407741546631),
 ('😎_SYM', 0.9668769240379333),
 ('🇺🇦_SYM', 0.9666495323181152),
 ('л_NOUN', 0.966334879398346),
 ('вроде_PRT', 0.9661213755607605),
 ('сергей_VERB', 0.9642270803451538),
 ('🤣🤣🤣_.', 0.9640722274780273)]

In [215]:
model_vk_emoji.wv.most_similar('😡_SYM')

[('диа_NOUN', 0.2571726441383362),
 ('авиаконверсия_NOUN', 0.2465270459651947),
 ('денюжка_NOUN', 0.2398979812860489),
 ('айдахо_NOUN', 0.23455603420734406),
 ('мартаг_NOUN', 0.23232384026050568),
 ('завахири_VERB', 0.22832253575325012),
 ('либи_NOUN', 0.22597184777259827),
 ('нерчинский_ADJ', 0.22368426620960236),
 ('камар_NOUN', 0.21868754923343658),
 ('‼_SYM', 0.21711719036102295)]

# Обучение на сырых данных ВК 

In [218]:
model_vk = wv(vector_size=300, min_count=3, window=5)

In [220]:
model_vk.build_vocab(true_vk_data)

In [221]:
len(model_vk.wv)

13687

In [223]:
model_vk.train(true_vk_data, total_examples=model_vk.corpus_count, epochs=1)

(349278, 394259)

In [234]:
model_vk.wv.most_similar('🤡')

[('найти', 0.9976010322570801),
 ('бот', 0.9975552558898926),
 ('елена', 0.9974233508110046),
 ('🤣🤣🤣', 0.9974158406257629),
 ('александр', 0.9973702430725098),
 ('нога', 0.9973556399345398),
 ('😉', 0.997337281703949),
 ('работа', 0.9973090291023254),
 ('♂️', 0.9972916841506958),
 ('её', 0.9972783327102661)]

# Подготовка датасета для анализа тональности  *(дополнительно)* 

In [20]:
negative = ['😡', '👿', '😾', '😤', '😠', '🤬', '💀', '☠️']

In [21]:
sarcastic = ['с', '🤡', '🙃', '🤢', '😈']  # Нужно дополнить

In [22]:
positive = ['😀', '😃', '😉']  # Нужно дополнить